<a href="https://colab.research.google.com/github/Ramyasaisriveni/Fmmllab2024/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1(ANS)

Increasing the Validation Set SizeMore Representative Sample: A larger validation set can provide a more reliable estimate of the model's performance since it better represents the overall distribution of the data.Stability: The accuracy metric tends to be more stable and less prone to fluctuations because the larger sample size reduces variance. This means you're more likely to get a consistent measure of accuracy across different runs.Less Training Data: The downside is that less data is available for training, which might lead to a slightly less optimized model, potentially reducing the accuracy of the model in practice. However, this effect is usually minor if the training set is still sufficiently large.Reducing the Validation Set SizeLess Representative: A smaller validation set might not capture the full variability of the data, leading to an accuracy metric that might be less reliable. It may either overestimate or underestimate the true performance of the model.More Variance: The accuracy might fluctuate more between different runs due to the smaller sample size, making it less stable as a performance measure.More Training Data: With more data available for training, the model might be slightly better optimized, but the improvement might be marginal if the reduction in validation data leads to poor evaluation accuracy.SummaryIncreasing validation set size: More reliable and stable accuracy estimates, but slightly less training data.Decreasing validation set size: Less reliable accuracy estimates with more variance, but slightly more training data.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Define percentages for validation set
validation_percentages = [0.1, 0.2, 0.3]  # 10%, 20%, 30%

# Train and evaluate model with different validation set sizes
for val_percent in validation_percentages:
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_percent, random_state=42)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions and evaluate accuracy
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    print(f"Validation set size: {int(val_percent * 100)}%")
    print(f"Validation accuracy: {accuracy:.4f}\n")

2(ANS)
The size of the training and validation sets can significantly impact how well the validation set's accuracy predicts the test set's accuracy. Here’s how:

### Larger Training Set:
- **Better Model Performance:** More training data generally helps the model learn better and generalize more effectively. This can lead to a model with improved performance on both the validation and test sets.
- **Impact on Validation Accuracy:** With a larger training set, the model may become more robust, leading to more reliable performance estimates from the validation set. However, if the validation set is small, the estimates of performance might still be noisy.

### Larger Validation Set:
- **More Reliable Estimates:** A larger validation set provides a more accurate estimate of the model’s performance on unseen data. This is because it is more representative of the overall data distribution.
- **Better Prediction of Test Accuracy:** With a larger validation set, the performance metric (e.g., accuracy) is less likely to be affected by random fluctuations and better reflects the model’s expected performance on the test set.

### Smaller Training Set:
- **Potential Overfitting:** A smaller training set might lead to overfitting, where the model performs well on the training set but poorly on the validation set. This can make the validation accuracy less reliable.
- **Impact on Validation Accuracy:** If the model is under-trained due to insufficient data, the validation accuracy might not accurately reflect the true performance on the test set.

### Smaller Validation Set:
- **Noisy Estimates:** A smaller validation set might not capture the full variability of the data, making the accuracy estimate more variable and less reliable.
- **Worse Prediction of Test Accuracy:** If the validation set is too small, it may not provide a good estimate of the model’s performance on the test set, leading to less reliable predictions of test accuracy.

### Summary
- **Larger Training Set:** Generally improves model performance and makes the validation accuracy more reliable.
- **Larger Validation Set:** Provides more accurate estimates of performance and better predicts test set accuracy.
- **Smaller Training Set:** Can lead to overfitting and unreliable validation accuracy.
- **Smaller Validation Set:** Results in less reliable accuracy estimates and poorer prediction of test set performance.

In practice, it’s crucial to find a balance that ensures a sufficiently large training set to train the model effectively and a sufficiently large validation set to provide a reliable estimate of performance.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Define training and validation set sizes
train_val_split = [0.2, 0.3, 0.4]  # Validation set sizes (20%, 30%, 40%)

# Split data into training+validation and test sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

for val_size in train_val_split:
    # Split the temporary data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size, random_state=42)

    # Initialize and train the model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Make predictions and evaluate accuracy on validation set
    y_val_pred = model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Make predictions and evaluate accuracy on test set
    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    print(f"Validation set size: {int(val_size * 100)}%")
    print(f"Validation accuracy: {val_accuracy:.4f}")
    print(f"Test accuracy: {test_accuracy:.4f}\n")

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

3(ANS)
A common practice in machine learning is to reserve around **20%** of the data for the validation set. This percentage strikes a balance between having enough data for both training and validation while ensuring the validation set provides a reliable estimate of model performance. Here’s why 20% is often used:

1. **Sufficient Size:** With 20% reserved for validation, you generally have enough data to get a meaningful estimate of the model’s performance, reducing variability and providing a stable measure of accuracy.
   
2. **Adequate Training Data:** This leaves 80% of the data for training, which is typically enough to train most models effectively without overfitting.

### Alternative Approaches

- **Cross-Validation:** If you’re concerned about the representativeness of a single validation set split, you can use k-fold cross-validation, where the data is split into k subsets. The model is trained k times, each time using a different subset as the validation set and the remaining data for training. This approach provides a more comprehensive evaluation of the model’s performance.

- **Training Set Size:** For very large datasets, even smaller percentages can be sufficient for validation because the absolute number of samples remains large. Conversely, for smaller datasets, you might need to reserve a slightly larger percentage to ensure robust performance evaluation.

In summary, 20% is a good starting point, but adjustments might be necessary depending on the dataset size and specific requirements of the problem.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Split data into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Further split the training+validation set into training and validation sets (20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions and evaluate accuracy on validation set
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

# Make predictions and evaluate accuracy on test set
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Validation set size: 20% of training+validation data")
print(f"Validation accuracy: {val_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1(ANS)

Load Dataset: Uses the Iris dataset.Initialize Model: Sets up the RandomForest model.Cross-Validation: Uses cross_val_score to perform 5-Fold Cross-Validation, which splits the data into 5 folds, trains and validates the model 5 times, and returns accuracy scores for each fold.Output: Prints the accuracy for each fold, the mean accuracy, and the standard deviation, providing a comprehensive view of the model’s performance.Averaging results from cross-validation provides a more robust assessment of model performance, reducing the potential bias from any single train-validation split.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform k-Fold Cross-Validation
scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')  # 5-Fold Cross-Validation

print(f"Cross-Validation accuracies: {scores}")
print(f"Mean accuracy: {scores.mean():.4f}")
print(f"Standard deviation: {scores.std():.4f}")

2(ANS)
Cross-validation typically provides a more reliable estimate of model performance on unseen data compared to a single train-validation split, but it does not directly estimate test set accuracy. Here’s how it works:

### Accuracy of Cross-Validation Estimates

- **Improved Reliability:** By averaging performance metrics across multiple folds, cross-validation gives a more stable estimate of how well the model generalizes to new data. This is due to the model being trained and validated on different subsets of the data, which helps mitigate the risk of overfitting to a specific train-validation split.

- **Model Selection:** Cross-validation is often used to compare different models or hyperparameters. It provides a more robust measure of performance, helping in choosing the best model configuration.

### Test Set Accuracy

- **Separate Validation:** Test set accuracy is evaluated on a completely separate dataset that was not used during any part of the model training or validation. This provides an unbiased estimate of the model's performance on truly unseen data.

- **Cross-Validation vs. Test Set:** While cross-validation estimates the model's performance on unseen data within the context of the given dataset, the test set provides a final check on how the model performs on completely new data.

### Combining Both Approaches

1. **Use Cross-Validation for Model Tuning:** Apply cross-validation to tune model hyperparameters and select the best model configuration. This helps ensure that the model generalizes well and is less likely to overfit.

2. **Test Set for Final Evaluation:** After selecting the best model using cross-validation, evaluate its performance on a separate test set to get an unbiased measure of its effectiveness on new, unseen data.

### Summary

- **Cross-Validation:** Provides a more reliable estimate of performance compared to a single validation split, but is still within the context of the original dataset.
- **Test Set:** Gives an unbiased final evaluation of the model’s performance on completely unseen data.

Cross-validation helps in assessing how well the model might perform on unseen data within the context of the available data, while the test set provides the final check for performance.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Split the data into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform k-Fold Cross-Validation on the training+validation set
cv_scores = cross_val_score(model, X_train_val, y_train_val, cv=5, scoring='accuracy')  # 5-Fold Cross-Validation

# Train the model on the entire training+validation set
model.fit(X_train_val, y_train_val)

# Evaluate the model on the test set
test_accuracy = model.score(X_test, y_test)

print(f"Cross-Validation accuracies: {cv_scores}")
print(f"Mean Cross-Validation accuracy: {cv_scores.mean():.4f}")
print(f"Standard deviation of Cross-Validation accuracy: {cv_scores.std():.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

3(ANS)
Effects of Number of FoldsFewer Folds (e.g., 2-5):Higher Variance: With fewer folds, each fold contains a larger portion of the data, and the model’s performance can be highly variable depending on how the data is split.Less Reliable: Fewer folds may not capture the variability in the data as well, leading to less reliable estimates of model performance.More Folds (e.g., 10):Lower Variance: More folds mean each fold is smaller, which helps in obtaining a more stable estimate of performance by averaging results from more different subsets of the data.More Reliable: More folds generally provide a more accurate estimate of model performance, as each data point is used for both training and validation multiple times.Trade-OffsComputational Cost: Increasing the number of folds requires more computational resources since the model is trained and validated more times. For example, 10-fold cross-validation involves training the model 10 times, which can be computationally expensive for large datasets or complex models.Bias-Variance Trade-Off: A higher number of folds reduces variance (by averaging over more splits) but may introduce more computational overhead. Conversely, fewer folds may lead to higher variance but less computational cost.Practical ConsiderationsTypical Choices: Common choices for the number of folds are 5 or 10. These values generally balance the trade-offs between computational cost and the reliability of the performance estimate.Dataset Size: For smaller datasets, more folds can be beneficial because it allows each sample to be used for both training and validation more times. For larger datasets, fewer folds might be sufficient.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Split the data into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Different numbers of folds to test
folds = [3, 5, 10]

for num_folds in folds:
    # Perform k-Fold Cross-Validation
    cv_scores = cross_val_score(model, X_train_val, y_train_val, cv=num_folds, scoring='accuracy')

    # Train the model on the entire training+validation set
    model.fit(X_train_val, y_train_val)

    # Evaluate the model on the test set
    test_accuracy = model.score(X_test, y_test)

    print(f"Number of folds: {num_folds}")
    print(f"Cross-Validation accuracies: {cv_scores}")
    print(f"Mean Cross-Validation accuracy: {cv_scores.mean():.4f}")
    print(f"Standard deviation of Cross-Validation accuracy: {cv_scores.std():.4f}")
    print(f"Test set accuracy: {test_accuracy:.4f}\n")

4(ANS)
Effects of Number of FoldsFewer Folds (e.g., 2-5):Higher Variance: With fewer folds, each fold contains a larger portion of the data, and the model’s performance can be highly variable depending on how the data is split.Less Reliable: Fewer folds may not capture the variability in the data as well, leading to less reliable estimates of model performance.More Folds (e.g., 10):Lower Variance: More folds mean each fold is smaller, which helps in obtaining a more stable estimate of performance by averaging results from more different subsets of the data.More Reliable: More folds generally provide a more accurate estimate of model performance, as each data point is used for both training and validation multiple times.Trade-OffsComputational Cost: Increasing the number of folds requires more computational resources since the model is trained and validated more times. For example, 10-fold cross-validation involves training the model 10 times, which can be computationally expensive for large datasets or complex models.Bias-Variance Trade-Off: A higher number of folds reduces variance (by averaging over more splits) but may introduce more computational overhead. Conversely, fewer folds may lead to higher variance but less computational cost.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load the dataset
data = load_iris()
X, y = data.data, data.target

# Split the data into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform k-Fold Cross-Validation on the training+validation set
cv_scores = cross_val_score(model, X_train_val, y_train_val, cv=5, scoring='accuracy')  # 5-Fold Cross-Validation

# Train the model on the entire training+validation set
model.fit(X_train_val, y_train_val)

# Evaluate the model on the test set
test_accuracy = model.score(X_test, y_test)

print(f"Cross-Validation accuracies: {cv_scores}")
print(f"Mean Cross-Validation accuracy: {cv_scores.mean():.4f}")
print(f"Standard deviation of Cross-Validation accuracy: {cv_scores.std():.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.